In [9]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pandas as pd
import pickle
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


from collections import Counter
c = Counter()


In [11]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Identifying the toxic word

### Bag of words

In [3]:
toxic_word_path='../data/vocab/negative_words.txt'

with open(toxic_word_path, 'r') as f:
    toxic_word_list = f.read().splitlines()
toxic_word_list = [line.strip() for line in toxic_word_list]

def bag_of_words(sentence, list=toxic_word_list):

    sentence_list = sentence.split()

    for word in sentence_list:
        if word in list:
            sentence = sentence.replace(word, "[MASK]")


    return sentence


In [5]:
bag_of_words('I fucking hate you')

'I [MASK] hate you'

### Logistic Regression

In [ ]:
!pip install scikit-learn
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [ ]:
X_train = corpus_tox + corpus_norm
y_train = [1] * len(corpus_tox) + [0] * len(corpus_norm)
pipe.fit(X_train, y_train)

In [ ]:
coefs = pipe[1].coef_[0]
coefs.shape

In [ ]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [ ]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

### Roberta

## Use the fine-tuned BERT to predict alternatives

### BERT Model: Random mask

### BERT Model: Targetted mask

## Ranking based on Metrics

In [ ]:

def ranker(similarity_scorer, toxicity_scorer, fluency_scorer, possible_sentences):

  max_score = -1

  detoxed_sentence = ''

  for sentence in possible_sentences:
    
    sent_score = similarity_scorer(sentence)*toxicity_scorer(sentence)*fluency_scorer(sentence)

    if sent_score > max_score:
      detoxed_sentence = sentence
    
  return sentence

## Pipeline

In [ ]:
def pipeline(masker, predictor, similarity_scorer, toxicity_scorer, fluency_scorer, ranker, input_sentence, model):

  # input_sentence is the sentence we wish to detoxify
  # masker inputs sentence as string, outputs sentence as string with [MASK] tokens
  # predictor inputs sentence with [MASK], returns a list of N sentences, with the masks replaced
  # similarity _scorer inputs 2 sentences and returns a value, how similar they are
  # toxicity_scorer inputs 1 sentence and returns a value, how toxic the sentence is
  # fluency_scorer, inputs 1 sentence and returns a value, how fluent the sentence is
  # ranker inputs a list of N sentences, and returns one sentence, the most appropriately detoxed sentence
  # model is the fine-tuned model used for the prediction task


  masked_sentence = masker(input_sentence)

  possible_sentences = predictor(masked_sentence, N, model)

  detoxed_sentence = ranker(similarity_scorer,toxicity_scorer,fluency_scorer, possible_sentences)

  return detoxed_sentence

